# Fairness Testing

Click the ipynb links to run examples in [Google Colab](https://colab.research.google.com/github/SelfExplainML/PiML-Toolbox/blob/main/examples/Example_fairness.ipynb).

## Stage 1: Data Generation

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

def sigmoid(beta, x):
    y = 1/(1 + np.exp(beta[0]+beta[1]*x))
    return y

def gen_data(seed=7):
    #simulation data generation
    np.random.seed(seed)
    N = 10000
    A = np.random.binomial(1,0.5, (N,1))
    B0 = np.random.binomial(1,0.5, (N,1))
    x1 = (0.3*A) + np.random.normal(0,0.5,(N,1))
    x20 = np.random.normal(0,0.5,(N,1))
    x2 = (x20 >0)*x20
    x30 = B0
    x3 = (x30 > 0)*1.0
    x40 = (np.random.lognormal(1,0.7,(N,1))-1)/2.0
    x4d = np.round(x40).astype(int)
    x4 = (x40 - np.mean(x40))/(2*np.std(x40))
    yl = x1 - sigmoid((0,2),x2) + x3/2 - x4 + np.random.normal(0,0.5,(N,1))
    y = np.reshape((yl>np.mean(yl))*1,(N,1))

    scaler = MinMaxScaler()
    scaler.fit(x1)
    x1d = np.round(scaler.transform(x1)*10000).astype(int)
    scaler.fit(x2)
    x2d = np.round(scaler.transform(x2)*10000).astype(int)
    x2f = (x2d > x1d)*x1d + (x2d<=x1d)*x2d
    v2 = np.random.binomial(1,0.15, (N,1))
    B = ((B0+v2)>1)*0 + ((B0+v2)<=1)*(B0+v2)
    x = np.hstack((x3,x1d,x2f,x4d,A,B))

    df = pd.DataFrame(data=x, columns = ['Mortgage', 'Balance', 'Amount Past Due', 'Credit Inquiry', 'Gender','Race'])
    df = pd.concat([df,pd.DataFrame(data=y, columns = ['Approved'])],axis =1)
    return df

df = gen_data(seed=7)

## Stage 2: Initialize an experiment, Load and Prepare data

In [ ]:
from piml import Experiment
exp = Experiment()

In [3]:
# Manually load data to piml
exp.data_loader(data=df)

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

<IPython.core.display.Javascript object>

,Mortgage,Balance,Amount Past Due,Credit Inquiry,Gender,Race,Approved
0,1.0,3635.0,482.0,2.0,0.0,0.0,0.0
1,0.0,6265.0,0.0,1.0,1.0,0.0,0.0
2,1.0,4432.0,0.0,4.0,0.0,1.0,0.0
3,1.0,3092.0,0.0,1.0,1.0,1.0,0.0
4,1.0,8025.0,135.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...
9995,1.0,4503.0,687.0,3.0,0.0,1.0,0.0
9996,1.0,5201.0,0.0,0.0,1.0,1.0,1.0
9997,0.0,5566.0,0.0,1.0,1.0,0.0,1.0
9998,1.0,3755.0,0.0,1.0,1.0,1.0,0.0


In [4]:
# Exclude these features one-by-one: "Mortgage", "Gender", "Race"(demographic variables); 
# Excluded features will show in grey color in the table.
exp.data_summary()

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

<IPython.core.display.Javascript object>

HTML(value='<link rel="stylesheet" href="//stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.…

In [5]:
# Prepare dataset with Test Ratio = 0.33
exp.data_prepare()

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

<IPython.core.display.Javascript object>

## Stage 3. Train Models

In [6]:
# Choose XGB2 with default settings, click run; 
# When training is finished, register the model.
exp.model_train()

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

<IPython.core.display.Javascript object>

## Stage 4. Fairness Testing

Procudures:
- First select a registered model, say GLM.

- Then, in the Group Setting Panel:
  - Set Add Category="Gender", Select "1.0" in reference; Select "0.0" in protected; Click on Add Button.
  - Set Add Category="Race", Select "1.0" in reference; Select "0.0" in protected; Click on Add Button. 

- Then Switch to the Metrics Panel:
  - Select a metric (default to AIR); Select the Favorable Threshold and Class for cutting the predicted response as favored or unfavored.

- Then Switch to Segmented Metrics:
  - Select the segment feature; Select the number of bins if it is a numerical feature; otherwise, it will be segmented automatically using its classes.

- Then Switch to Thresholding:
   - Select a fairness metric (default to AIR); Select a performance metric (default to ACC); Select the Favorable Threshold and Class for cutting the predicted response as favored or unfavored. Red dashed line shows the threshold.

In [7]:
exp.model_fairness()

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>